In [1]:
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import tensorflow as tf

from keras.datasets import boston_housing
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


# Boston Housing

> loss='mse', optimizer='rmsprop',metrics=['mae']

> ex) mae가 0.5 -> 실제와 예측 차이:0.5 -> 500달러 차이

> 케라스, 텐서플로우

In [2]:
(trainData,trainTargets), (testData,testTargets)=boston_housing.load_data()

In [3]:
print(trainData.shape, testData.shape)

(404, 13) (102, 13)


In [4]:
xTrn = trainData[:,:-1]
yTrn = trainData[:,[-1]]
xTst = testData[:,:-1]
yTst = testData[:,[-1]]

In [5]:
scaler = StandardScaler()
xTrn = scaler.fit_transform(xTrn)
xTst = scaler.transform(xTst)

## Tensorflow

In [7]:
tf.reset_default_graph()

In [8]:
x=tf.placeholder(tf.float32,[None,12])
y=tf.placeholder(tf.float32,[None,1])

w1=tf.get_variable("w1",shape=[12,256], initializer=tf.contrib.layers.xavier_initializer())
b1=tf.Variable(tf.random_normal([256]))
l1=tf.nn.relu(tf.matmul(x,w1)+b1)

w2=tf.get_variable("w2",shape=[256,256], initializer=tf.contrib.layers.xavier_initializer())
b2=tf.Variable(tf.random_normal([256]))
l2=tf.nn.relu(tf.matmul(l1,w2)+b2)

w3=tf.get_variable("w3",shape=[256,256], initializer=tf.contrib.layers.xavier_initializer())
b3=tf.Variable(tf.random_normal([256]))
l3=tf.nn.relu(tf.matmul(l2,w3)+b3)

w4=tf.get_variable("w4",shape=[256,64], initializer=tf.contrib.layers.xavier_initializer())
b4=tf.Variable(tf.random_normal([64]))
l4=tf.nn.relu(tf.matmul(l3,w4)+b4)

w5=tf.get_variable("w5",shape=[64,1], initializer=tf.contrib.layers.xavier_initializer())
b5=tf.Variable(tf.random_normal([1]))

hf=tf.matmul(l4,w5)+b5

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [9]:
keep_prob = tf.placeholder(tf.float32)
rmse=tf.sqrt(tf.reduce_mean(tf.square(hf-y)))
optimizer=tf.train.AdamOptimizer(0.001).minimize(rmse)

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(10000):
    rm_, _ = sess.run([rmse, optimizer],
                    feed_dict={x:xTrn, y:yTrn, keep_prob:0.7})
    if epoch % 1000 == 0:
        print('rmse : %f' % rm_)
pred, loss_v = sess.run([hf, rmse], feed_dict={x:xTst, y:yTst})
print("result : %.3f" % loss_v)

rmse : 17.058138
rmse : 1.057937
rmse : 0.725401
rmse : 0.586818
rmse : 0.477260
rmse : 0.413931
rmse : 0.340204
rmse : 0.329072
rmse : 0.251491
rmse : 0.258049
result : 4.823


In [15]:
for i, (p, r) in enumerate(zip(pred, yTst)):
    print(p, r)
    if i >= 10 : break

[16.319784] [29.05]
[16.004847] [16.21]
[8.9776325] [9.74]
[9.524349] [5.5]
[11.891737] [8.44]
[12.754826] [13.59]
[3.9335237] [4.61]
[9.003914] [7.51]
[16.276075] [17.93]
[11.210312] [14.36]
[32.858414] [10.63]


## Keras

In [17]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

In [18]:
model = Sequential()
model.add(Dense(12, input_dim=12, activation='relu'))
model.add(Dense(144, activation='relu'))
model.add(Dense(144, activation='relu'))
model.add(Dense(288, activation='relu'))
model.add(Dense(288, activation='relu'))
model.add(Dense(144, activation='relu'))
model.add(Dense(1))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [22]:
#loss='mse', optimizer='rmsprop',metrics=['mae']
model.compile(loss="mae",
             optimizer='rmsprop',
             metrics=['mae'])

In [29]:
# from keras.callbacks import EarlyStopping
# es = EarlyStopping(monitor='val_loss', patience=100)

In [34]:
model.fit(xTrn,yTrn,epochs=200,batch_size=100)

Epoch 1/200
404/404 [==============================] - 0s 39us/step - loss: 1.2425 - mae: 1.2425
Epoch 2/200
404/404 [==============================] - 0s 42us/step - loss: 1.1967 - mae: 1.1967
Epoch 3/200
404/404 [==============================] - 0s 47us/step - loss: 1.2365 - mae: 1.2365
Epoch 4/200
404/404 [==============================] - 0s 44us/step - loss: 1.3760 - mae: 1.3760
Epoch 5/200
404/404 [==============================] - 0s 42us/step - loss: 1.3329 - mae: 1.3329
Epoch 6/200
404/404 [==============================] - 0s 42us/step - loss: 1.0653 - mae: 1.0653
Epoch 7/200
404/404 [==============================] - 0s 47us/step - loss: 1.1903 - mae: 1.1903
Epoch 8/200
404/404 [==============================] - 0s 42us/step - loss: 1.2427 - mae: 1.2427
Epoch 9/200
404/404 [==============================] - 0s 44us/step - loss: 1.1754 - mae: 1.1754
Epoch 10/200
404/404 [==============================] - 0s 44us/step - loss: 1.1821 - mae: 1.1821
Epoch 11/200
404/404 [=======

404/404 [==============================] - 0s 42us/step - loss: 1.2114 - mae: 1.2114
Epoch 85/200
404/404 [==============================] - 0s 42us/step - loss: 0.9877 - mae: 0.9877
Epoch 86/200
404/404 [==============================] - 0s 37us/step - loss: 0.9464 - mae: 0.9464
Epoch 87/200
404/404 [==============================] - 0s 40us/step - loss: 0.8774 - mae: 0.8774
Epoch 88/200
404/404 [==============================] - 0s 47us/step - loss: 0.9492 - mae: 0.9492
Epoch 89/200
404/404 [==============================] - 0s 44us/step - loss: 0.9220 - mae: 0.9220
Epoch 90/200
404/404 [==============================] - 0s 37us/step - loss: 1.0313 - mae: 1.0313
Epoch 91/200
404/404 [==============================] - 0s 39us/step - loss: 0.8713 - mae: 0.8713
Epoch 92/200
404/404 [==============================] - 0s 35us/step - loss: 0.7817 - mae: 0.7817
Epoch 93/200
404/404 [==============================] - 0s 37us/step - loss: 0.9560 - mae: 0.9560
Epoch 94/200
404/404 [===========

404/404 [==============================] - 0s 35us/step - loss: 0.8896 - mae: 0.8896
Epoch 167/200
404/404 [==============================] - 0s 35us/step - loss: 0.8513 - mae: 0.8513
Epoch 168/200
404/404 [==============================] - 0s 35us/step - loss: 0.8282 - mae: 0.8282
Epoch 169/200
404/404 [==============================] - 0s 35us/step - loss: 0.8816 - mae: 0.8816
Epoch 170/200
404/404 [==============================] - 0s 35us/step - loss: 1.0091 - mae: 1.0091
Epoch 171/200
404/404 [==============================] - 0s 37us/step - loss: 1.1678 - mae: 1.1678
Epoch 172/200
404/404 [==============================] - 0s 35us/step - loss: 0.7385 - mae: 0.7385
Epoch 173/200
404/404 [==============================] - 0s 40us/step - loss: 0.7565 - mae: 0.7565
Epoch 174/200
404/404 [==============================] - 0s 40us/step - loss: 0.9788 - mae: 0.9788
Epoch 175/200
404/404 [==============================] - 0s 37us/step - loss: 0.7262 - mae: 0.7262
Epoch 176/200
404/404 [=

In [35]:
model.evaluate(xTst,yTst)

102/102 [==============================] - 0s 39us/step


[2.7817002675112557, 2.781700372695923]

In [36]:
for i, (p, r) in enumerate(zip(pred, yTst)):
    print(p, r)
    if i >= 10 : break

[16.319784] [29.05]
[16.004847] [16.21]
[8.9776325] [9.74]
[9.524349] [5.5]
[11.891737] [8.44]
[12.754826] [13.59]
[3.9335237] [4.61]
[9.003914] [7.51]
[16.276075] [17.93]
[11.210312] [14.36]
[32.858414] [10.63]
